In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# 10-fold subject-independent cross-validation of the MLP-SI model
# Using the MLP-SI model, train&test on 10 folds (every time 2 test subjects and 2 validation subjects) 
# Then save the results by subjects. 
# 1. cell - evaluate on validation partition
# 2. cell - evaluate on testing partition
#####################################################################################

In [ ]:
###############################################################################################################
# Evaluate on validation partition
###############################################################################################################

import numpy as np
import time
import glob
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from geoutils import radToDeg, degToRad
from evalutils import norm_Y, inv_norm_Y
from postprocessingutils import save_predictions_and_eval2

N_folds = 10

SEGMENT_LEN = 300 # for evaluation (local cca)

np.random.seed(37) # for reproducibility
TE_folder = 'TrainingExamples_16kHz'
unique_srt_VIDs = unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

###
# Learning settings
epochs = 1000
dropout = 0.

########################################################################
# For given subject SID (PID02) get corresponding VIDs
def get_subjects_VIDs(SIDs):
    result_VIDs = []
    for SID in SIDs:
        # Take VIDs from both tasks, for this subject
        result_VIDs.append(  SID + 'Task2' )
        result_VIDs.append(  SID + 'Task3' )
    return result_VIDs

# For given subjects get corresponding indicies into the feature set & also their counts (per VID)
def get_subjects_indicies(SIDs):
    indicies = []
    indicies_cnts = []
    for SID in SIDs:
        # Take indicies corresponding to both tasks, for this subject
        a = np.argwhere(all_srt_VIDs == SID + 'Task2')[:,0]
        b = np.argwhere(all_srt_VIDs == SID + 'Task3')[:,0]
        indicies.extend( a )
        indicies.extend( b )
        indicies_cnts.append( len(a) )
        indicies_cnts.append( len(b) )
    return indicies, indicies_cnts
# print frameCnts[26] + frameCnts[27] + frameCnts[0] + frameCnts[1] # checks
# print len(get_subjects_indicies(['PID20', 'PID02']))
########################################################################

save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/cvVal/'
model_checkpoint_path_prefix = './ModelCheckpoints/MLP_SI/cvVal/'

PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]
N_targets = PF.shape[1]
###########
# Target (Y) normalisation, into range 0-1 according to constraints
PF = norm_Y(PF)
print "Targets (Y) are TRANSFORMED to 0-1 range"

tuning_types = [
    '1_35_AF13',
    '1_35_AF26', 
    '1_35_AF52', 
    '1_35_AF78'
]
AF_types = [
    'AF_MFCC13_norm',
    'AF_logFB26_norm',
    'AF_logFB52_norm',
    'AF_logFB78_norm'
]

for tuning_type, AF_type in zip(tuning_types, AF_types):
    
    print AF_type, tuning_type

    AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
    N_features = AF.shape[1]

    #######################
    # Load validation data
    dd = np.load('./../Dataset/'+TE_folder+'/Results/MLP_SI/XXXval_' + tuning_type + '.npz')
    best_N_hl = int(dd['best_N_hl'])
    best_N_hu = int(dd['best_N_hu'])    
    print "\tOptimal number of hidden layers / hidden units: ", best_N_hl, " / ", best_N_hu

    #######################################################
    # Dataset split

    N_test_SIDs = 2
    N_val_SIDs = 2
    N_SIDs = 19
    N_train_SIDs = N_SIDs - N_test_SIDs - N_val_SIDs
    print "Dataset split in terms of subjects (train/val/test): ", 100.*N_train_SIDs/N_SIDs, "/", 100.*N_val_SIDs/N_SIDs, "/", 100.*N_test_SIDs/N_SIDs, "%"
    print 

    # Randomise the dataset split
    permI = np.random.permutation(N_SIDs)

    for fold_i in range(N_folds):

        print permI
        train_SIDs_mask = permI[:N_train_SIDs] 
        val_SIDs_mask =   permI[N_train_SIDs:N_train_SIDs+N_val_SIDs]
        test_SIDs_mask =  permI[N_train_SIDs+N_val_SIDs:] 

        print "Train SIDs",      unique_srt_SIDs[train_SIDs_mask]
        print "Valid SIDs",      unique_srt_SIDs[val_SIDs_mask]
        print "Testi SIDs",      unique_srt_SIDs[test_SIDs_mask]

        print "Train SIDs mask", train_SIDs_mask
        print "Valid SIDs mask", val_SIDs_mask
        print "Testi SIDs mask", test_SIDs_mask

        train_VIDs = get_subjects_VIDs(unique_srt_SIDs[train_SIDs_mask])
        val_VIDs   = get_subjects_VIDs(unique_srt_SIDs[val_SIDs_mask])
        test_VIDs  = get_subjects_VIDs(unique_srt_SIDs[test_SIDs_mask])

        print "Train VIDs", train_VIDs
        print "Valid VIDs", val_VIDs
        print "Testi VIDs", test_VIDs

        train_mask, train_VIDs_ind_cnts = get_subjects_indicies(unique_srt_SIDs[train_SIDs_mask])
        val_mask, val_VIDs_ind_cnts     = get_subjects_indicies(unique_srt_SIDs[val_SIDs_mask])
        test_mask, test_VIDs_ind_cnts   = get_subjects_indicies(unique_srt_SIDs[test_SIDs_mask])
        # print test_VIDs_ind_cnts

        print "Train/val/test set sizes: ", len(train_mask), "/", len(val_mask), "/", len(test_mask), " = ", len(train_mask) + len(val_mask) + len(test_mask)
        print "Dataset split in terms of #examples (train/val/test): ", 100.*len(train_mask)/len(all_srt_VIDs), "/", 100.*len(val_mask)/len(all_srt_VIDs), "/", 100.*len(test_mask)/len(all_srt_VIDs), "%"
        print 

        permI = np.roll(permI, 2) # ROTATE INDICES - FOR NEXT DATASET SPLIT

        #######################################################
        # Training

        Y_train = PF[train_mask]
        Y_val   = PF[val_mask]
        #Y_test  = PF[test_mask]
        X_train = AF[train_mask]
        X_val   = AF[val_mask]
        #X_test  = AF[test_mask]
        train_batch_size = len(X_train)
        val_batch_size = len(X_val)
        #test_batch_size = len(X_test)

        st = time.time()

        ##########################
        # Final train & test

        # Create model
        model = Sequential()
        model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform', input_dim=N_features))
        #model.add(Dropout(dropout))
        for _ in range(1, best_N_hl):
            model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform'))
            #model.add(Dropout(dropout))
        model.add(Dense(N_targets, activation='sigmoid'))

        model.compile(loss='mean_squared_error', optimizer=Adam())
        #print model.summary()
        #print "#parameters: ", model.count_params()

        early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1) # stop after 10 epochs without improvement in val_acc

        # Checkpoint model weights and the model itself: at each epoch
        model_checkpoint_name = 'm_{:}_{:02d}'.format(tuning_type, i) + '_{epoch:04d}_{loss:.4f}_{val_loss:.4f}.hdf5'
        model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + model_checkpoint_name, monitor='val_loss', 
                                           verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

        hist = model.fit(X_train, Y_train, epochs=epochs, batch_size=train_batch_size, 
                   validation_data = (X_val, Y_val), verbose=0, callbacks=[early_stop, model_checkpoint])

        #######################################################
        # Testing & evaluation & saving results

        Y_val_pred = model.predict(X_val, batch_size=val_batch_size, verbose=1)

        # 1. val subject
        cnt = val_VIDs_ind_cnts[0] + val_VIDs_ind_cnts[1]

        # Avoid evaluation of PID20 twice
        if fold_i == N_folds - 1 and val_VIDs[0][:5] == 'PID20':
            print "Avoiding evaluation of PID20 twice!"
        else:
            save_predictions_and_eval2(save_results_path_prefix + 'val_' + tuning_type + '_' + val_VIDs[0][:5], 
                            X_val[:cnt], Y_val[:cnt], Y_val_pred[:cnt], 'MLP_SI', SEGMENT_LEN, val_VIDs[:2], 
                            val_VIDs_ind_cnts[:2], N_params=model.count_params(), N_epochs=len(hist.history['loss']))    

        # 2. val subject
        save_predictions_and_eval2(save_results_path_prefix + 'val_' + tuning_type + '_' + val_VIDs[2][:5], 
                        X_val[cnt:], Y_val[cnt:], Y_val_pred[cnt:], 'MLP_SI', SEGMENT_LEN, val_VIDs[2:], 
                        val_VIDs_ind_cnts[2:], N_params=model.count_params(), N_epochs=len(hist.history['loss']))    

        print "\tTime taken: ", time.time()-st, (time.time()-st)/60.



In [ ]:
###############################################################################################################
# Evaluate on testing partition
###############################################################################################################

import numpy as np
import time
import glob
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from geoutils import radToDeg, degToRad
from evalutils import norm_Y, inv_norm_Y
from postprocessingutils import save_predictions_and_eval2

N_folds = 10

SEGMENT_LEN = 300 # for evaluation (local cca)
AF_type = 'AF_logFB26_norm'

np.random.seed(37) # for reproducibility
TE_folder = 'TrainingExamples_16kHz'
unique_srt_VIDs = unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

###
tuning_type = '1_35_AF26'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/cvTest/'
model_checkpoint_path_prefix = './ModelCheckpoints/MLP_SI/cvTest/'

PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]
N_targets = PF.shape[1]
###########
# Target (Y) normalisation, into range 0-1 according to constraints
PF = norm_Y(PF)
print "Targets (Y) are TRANSFORMED to 0-1 range"

AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
N_features = AF.shape[1]

#######################
# Load validation data
dd = np.load('./../Dataset/'+TE_folder+'/Results/MLP_SI/XXXval_' + tuning_type + '.npz')
best_N_hl = int(dd['best_N_hl'])
best_N_hu = int(dd['best_N_hu'])    
print "\tOptimal number of hidden layers / hidden units: ", best_N_hl, " / ", best_N_hu


# Learning settings
epochs = 1000
dropout = 0.

########################################################################
# For given subject SID (PID02) get corresponding VIDs
def get_subjects_VIDs(SIDs):
    result_VIDs = []
    for SID in SIDs:
        # Take VIDs from both tasks, for this subject
        result_VIDs.append(  SID + 'Task2' )
        result_VIDs.append(  SID + 'Task3' )
    return result_VIDs

# For given subjects get corresponding indicies into the feature set & also their counts (per VID)
def get_subjects_indicies(SIDs):
    indicies = []
    indicies_cnts = []
    for SID in SIDs:
        # Take indicies corresponding to both tasks, for this subject
        a = np.argwhere(all_srt_VIDs == SID + 'Task2')[:,0]
        b = np.argwhere(all_srt_VIDs == SID + 'Task3')[:,0]
        indicies.extend( a )
        indicies.extend( b )
        indicies_cnts.append( len(a) )
        indicies_cnts.append( len(b) )
    return indicies, indicies_cnts
# print frameCnts[26] + frameCnts[27] + frameCnts[0] + frameCnts[1] # checks
# print len(get_subjects_indicies(['PID20', 'PID02']))
########################################################################

#######################################################
# Dataset split

N_test_SIDs = 2
N_val_SIDs = 2
N_SIDs = 19
N_train_SIDs = N_SIDs - N_test_SIDs - N_val_SIDs
print "Dataset split in terms of subjects (train/val/test): ", 100.*N_train_SIDs/N_SIDs, "/", 100.*N_val_SIDs/N_SIDs, "/", 100.*N_test_SIDs/N_SIDs, "%"
print 

# Randomise the dataset split
permI = np.random.permutation(N_SIDs)

for fold_i in range(N_folds):
    
    print permI
    train_SIDs_mask = permI[:N_train_SIDs] 
    val_SIDs_mask =   permI[N_train_SIDs:N_train_SIDs+N_val_SIDs]
    test_SIDs_mask =  permI[N_train_SIDs+N_val_SIDs:] 
    
    print "Train SIDs",      unique_srt_SIDs[train_SIDs_mask]
    print "Valid SIDs",      unique_srt_SIDs[val_SIDs_mask]
    print "Testi SIDs",      unique_srt_SIDs[test_SIDs_mask]

    print "Train SIDs mask", train_SIDs_mask
    print "Valid SIDs mask", val_SIDs_mask
    print "Testi SIDs mask", test_SIDs_mask

    train_VIDs = get_subjects_VIDs(unique_srt_SIDs[train_SIDs_mask])
    val_VIDs   = get_subjects_VIDs(unique_srt_SIDs[val_SIDs_mask])
    test_VIDs  = get_subjects_VIDs(unique_srt_SIDs[test_SIDs_mask])

    print "Train VIDs", train_VIDs
    print "Valid VIDs", val_VIDs
    print "Testi VIDs", test_VIDs

    train_mask, train_VIDs_ind_cnts = get_subjects_indicies(unique_srt_SIDs[train_SIDs_mask])
    val_mask, val_VIDs_ind_cnts     = get_subjects_indicies(unique_srt_SIDs[val_SIDs_mask])
    test_mask, test_VIDs_ind_cnts   = get_subjects_indicies(unique_srt_SIDs[test_SIDs_mask])
    # print test_VIDs_ind_cnts

    print "Train/val/test set sizes: ", len(train_mask), "/", len(val_mask), "/", len(test_mask), " = ", len(train_mask) + len(val_mask) + len(test_mask)
    print "Dataset split in terms of #examples (train/val/test): ", 100.*len(train_mask)/len(all_srt_VIDs), "/", 100.*len(val_mask)/len(all_srt_VIDs), "/", 100.*len(test_mask)/len(all_srt_VIDs), "%"
    print 

    permI = np.roll(permI, 2) # ROTATE INDICES - FOR NEXT DATASET SPLIT

    #######################################################
    # Training

    Y_train = PF[train_mask]
    Y_val   = PF[val_mask]
    Y_test  = PF[test_mask]
    X_train = AF[train_mask]
    X_val   = AF[val_mask]
    X_test  = AF[test_mask]
    train_batch_size = len(X_train)
    val_batch_size = len(X_val)
    test_batch_size = len(X_test)
    
    st = time.time()

    ##########################
    # Final train & test

    # Create model
    model = Sequential()
    model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform', input_dim=N_features))
    #model.add(Dropout(dropout))
    for _ in range(1, best_N_hl):
        model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform'))
        #model.add(Dropout(dropout))
    model.add(Dense(N_targets, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer=Adam())
    #print model.summary()
    #print "#parameters: ", model.count_params()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1) # stop after 10 epochs without improvement in val_acc

    # Checkpoint model weights and the model itself: at each epoch
    model_checkpoint_name = 'm_{:02d}'.format(i) + '_{epoch:04d}_{loss:.4f}_{val_loss:.4f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + model_checkpoint_name, monitor='val_loss', 
                                       verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    hist = model.fit(X_train, Y_train, epochs=epochs, batch_size=train_batch_size, 
               validation_data = (X_val, Y_val), verbose=0, callbacks=[early_stop, model_checkpoint])

    #######################################################
    # Testing & evaluation & saving results
    
    Y_test_pred = model.predict(X_test, batch_size=test_batch_size, verbose=1)
    
    # 1. test subject
    cnt = test_VIDs_ind_cnts[0] + test_VIDs_ind_cnts[1]
    
    # Avoid evaluation of PID23 twice
    if fold_i == N_folds - 1 and test_VIDs[0][:5] == 'PID23':
        print "Avoiding evaluation of PID23 twice!"
    else:
        save_predictions_and_eval2(save_results_path_prefix + 'test_' + test_VIDs[0][:5], 
                        X_test[:cnt], Y_test[:cnt], Y_test_pred[:cnt], 'MLP_SI', SEGMENT_LEN, test_VIDs[:2], 
                        test_VIDs_ind_cnts[:2], N_params=model.count_params(), N_epochs=len(hist.history['loss']))    

    # 2. test subject
    save_predictions_and_eval2(save_results_path_prefix + 'test_' + test_VIDs[2][:5], 
                    X_test[cnt:], Y_test[cnt:], Y_test_pred[cnt:], 'MLP_SI', SEGMENT_LEN, test_VIDs[2:], 
                    test_VIDs_ind_cnts[2:], N_params=model.count_params(), N_epochs=len(hist.history['loss']))    

    print "\tTime taken: ", time.time()-st, (time.time()-st)/60.

